In [1]:
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import wait 
import dask
from dask_saturn import SaturnCluster
from dask.distributed import Client

import matplotlib.pyplot as plt
import json
import datetime
import re

import bokeh as bk
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model


In [2]:
cluster = SaturnCluster()
client = Client(cluster)
client.restart()

INFO:dask-saturn:Cluster is ready
INFO:dask-saturn:Registering default plugins
INFO:dask-saturn:{'tcp://192.168.111.195:40543': {'status': 'repeat'}, 'tcp://192.168.157.131:40677': {'status': 'repeat'}, 'tcp://192.168.164.3:36247': {'status': 'repeat'}, 'tcp://192.168.253.131:39053': {'status': 'repeat'}, 'tcp://192.168.72.67:43979': {'status': 'repeat'}}


Client Scheduler: tcp://d-steph-college-scorecard-proj-f647d66a9d5341e1bf4679ed4dc68db3.main-namespace:8786 Dashboard: https://d-steph-college-scorecard-proj-f647d66a9d5341e1bf4679ed4dc68db3.internal.saturnenterprise.io,Cluster Workers: 5 Cores: 80 Memory: 637.50 GB


In [3]:
%%time

import s3fs
s3 = s3fs.S3FileSystem(anon=True)
s3fpath = 's3://saturn-public-data/college-scorecard/Most-Recent-Cohorts-Field-of-Study.csv'

major = dd.read_csv(
    s3fpath,
    usecols = ['UNITID','OPEID6','INSTNM','CONTROL','MAIN','CIPCODE',
               'CIPDESC','CREDLEV','CREDDESC','EARN_MDN_HI_2YR'],
    storage_options={'anon': True},
    dtype = 'object',
    na_values = ['PrivacySuppressed'],
    assume_missing=False
)

s3fpath2 = 's3://saturn-public-data/college-scorecard/Most-Recent-Cohorts-All-Data-Elements.csv'

inst = dd.read_csv(
    s3fpath2,
    storage_options={'anon': True},
    dtype = 'object',
    na_values = ['PrivacySuppressed', -2],
    assume_missing=False
)

# target: MD_EARN_WNE_P8

CPU times: user 407 ms, sys: 40.7 ms, total: 448 ms
Wall time: 1.68 s


# Model concept

Predict income x years in future for graduates?
Median Earnings 10 Years after Matriculation

given: x features of college, choose a major, then return median 10 years out.

Feature ideas:
* incoming sat/act
* admission rate
* college type (private/public/nonprofit)
* annual tuition


Deploy model, then give an interpretability dashboard?


## Minimal MVP Model

In [4]:
inst2 = inst[['STABBR','HIGHDEG','REGION','LOCALE','CCSIZSET','ADM_RATE',
      'ADM_RATE_ALL','SAT_AVG','SAT_AVG_ALL','UGDS','NPT4_PUB','NPT4_PRIV',
      'TUITIONFEE_IN','TUITIONFEE_OUT','PFTFAC','MEDIAN_HH_INC',
     'UNITID','OPEID','OPEID6','INSTNM','CITY','ZIP']]

In [5]:
newdf2 = dd.merge(
    major,
    inst2,
    how="left",
    on=['UNITID', 'OPEID6', 'INSTNM'],
    suffixes=("_maj", "_inst")
)

In [6]:
newdf2 = newdf2.repartition(30)

In [7]:
newdf2.columns

Index(['UNITID', 'OPEID6', 'INSTNM', 'CONTROL', 'MAIN', 'CIPCODE', 'CIPDESC',
       'CREDLEV', 'CREDDESC', 'EARN_MDN_HI_2YR', 'STABBR', 'HIGHDEG', 'REGION',
       'LOCALE', 'CCSIZSET', 'ADM_RATE', 'ADM_RATE_ALL', 'SAT_AVG',
       'SAT_AVG_ALL', 'UGDS', 'NPT4_PUB', 'NPT4_PRIV', 'TUITIONFEE_IN',
       'TUITIONFEE_OUT', 'PFTFAC', 'MEDIAN_HH_INC', 'OPEID', 'CITY', 'ZIP'],
      dtype='object')

In [8]:
def killna(df):
    return df[df.EARN_MDN_HI_2YR.notna()]

def make_feat(df):
    df['deg_level'] = ['ug' if x in ["Bachelor's Degree",
      "Associate's Degree",
      'Undergraduate Certificate or Diploma',
      'Post-baccalaureate Certificate'] else 'grad' for x in df['CREDLEV']]

    df['tuition'] = df['NPT4_PUB'].fillna(0).astype('float64') + df['NPT4_PRIV'].fillna(0).astype('float64')
    df['UGDS'] = df['UGDS'].fillna(0).astype(int)
    df['TUITIONFEE_IN'] = df['TUITIONFEE_IN'].fillna(0).astype('float64')
    df['TUITIONFEE_OUT'] = df['TUITIONFEE_OUT'].fillna(0).astype('float64')
    df['PFTFAC'] = df['PFTFAC'].fillna(0).astype('float64')
    df['ADM_RATE_ALL'] = df['ADM_RATE_ALL'].fillna(0).astype('float64')
    df['MEDIAN_HH_INC'] = df['MEDIAN_HH_INC'].fillna(0).astype('float64')
    df['SAT_AVG_ALL'] = df['SAT_AVG_ALL'].fillna(0).astype(int)
    df['cip_cat'] = df['CIPCODE'].astype(str).str[0:3]

    return df

def clean_cipcode(df):
    valcounts = df['CIPDESC'].value_counts().reset_index()
    valcounts.columns = ['CIPDESC', 'COUNT']

    valcounts['CIPDESC_new'] = [series['CIPDESC'] if series['COUNT'] > 5 else "TooSmall" for (index, series) in valcounts.iterrows()]

    df = dd.merge(
        df,
        valcounts,
        how="left",
        on=['CIPDESC'],
        suffixes=("_orig", "_new")
    )
    return(df)

@dask.delayed
def process_data(df):
    df = killna(df)
    df = make_feat(df)
    df = clean_cipcode(df)
    return(df)

@dask.delayed
def split_data(df):
    
    X = df[['SAT_AVG_ALL','CREDDESC', 'CIPDESC_new',
            'CONTROL', 'REGION', 'tuition', 'LOCALE', 'ADM_RATE_ALL']]
    y = df[['EARN_MDN_HI_2YR']]
    
    return [X, y]

In [9]:
data = process_data(newdf2)

In [10]:
#outdata = data.compute()

In [11]:
#s3fpath = 's3://saturn-public-data/college-scorecard/cleaned_merged.csv'

#outdata.to_csv(s3fpath)

In [12]:
X, y = split_data(data).compute()

In [47]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61779 entries, 0 to 61778
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SAT_AVG_ALL   61779 non-null  int64  
 1   CREDDESC      61779 non-null  object 
 2   CIPDESC_new   61779 non-null  object 
 3   CONTROL       61779 non-null  object 
 4   REGION        57808 non-null  object 
 5   tuition       61779 non-null  float64
 6   LOCALE        57808 non-null  object 
 7   ADM_RATE_ALL  61779 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 4.2+ MB


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [49]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import KBinsDiscretizer

enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
naimp = SimpleImputer(fill_value='Unknown', strategy = 'constant')
imp = IterativeImputer(max_iter=10, random_state=0, initial_strategy='mean', add_indicator = True)
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')


In [50]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [('onehot', enc, ['CONTROL','CREDDESC', 'CIPDESC_new','REGION', 'LOCALE']),
    ('impute', imp, ['SAT_AVG_ALL', 'ADM_RATE_ALL'])], 
    remainder='passthrough' 
)

pipe = Pipeline(steps=[('coltrans', ct), ('linear', linear_model.LinearRegression())])
pipe = pipe.fit(X_train, y_train)

In [51]:
predictions = pipe.predict(X_test)
print(predictions)

[[28715.61135864]
 [47063.00680542]
 [28999.94668579]
 ...
 [42169.38534546]
 [24360.74850464]
 [58243.61297607]]


In [52]:
pipe.score(X_train,y_train)

0.7292365371773282

In [53]:
pipe.score(X_test,y_test)

0.7167047736138037

In [54]:
newdata3 = X_test.copy()
newdata3['truth'] = y_test.astype('int')
newdata3['pred'] = predictions

newdata3['pred'] = [0 if x < 0 else x for x in newdata3['pred']]


In [66]:
newdata3['CONTROL'].value_counts()

Public                 5056
Private, nonprofit     2574
Private, for-profit    1631
Foreign                   6
Name: CONTROL, dtype: int64

In [67]:
newdf = pd.DataFrame([[800, 'Bachelors Degree', 'English Language and Literature, General.', 'Private, nonprofit', '1', 15000, '11', .3]],
columns = ['SAT_AVG_ALL','CREDDESC', 'CIPDESC_new','CONTROL', 'REGION', 'tuition', 'LOCALE', 'ADM_RATE_ALL'])

pipe.predict(newdf)

array([[32361.04202271]])

In [55]:
from bokeh.models import NumeralTickFormatter
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral6
import time
from datetime import datetime as dt
from bokeh.plotting import figure
from bokeh.models import Span, Label
from bokeh.palettes import Viridis256, Cividis256, Turbo256


In [56]:
def plot_earn(df, groupvar):
    output_notebook()

    p = figure(title="Predicted Earnings", 
               y_axis_label='True', 
               x_axis_label='Predicted', 
               width=750, 
               height = 400)

    sourcedt = df 

    states = sourcedt[groupvar].fillna("NA").unique().tolist()

    p.circle(y='truth', x='pred', size=5, fill_alpha = .75,
             fill_color=factor_cmap(groupvar, 
                                    palette=Spectral6, 
                                    factors=states), 
             legend_group=groupvar,
             source=sourcedt)

    p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0.0")
    p.xaxis[0].formatter = NumeralTickFormatter(format="$0,0.0")


    show(p)

In [57]:
dt1 = newdata3[(newdata3['CONTROL'] != 'Public')]# | (newdata3['REGION'] == '9')]

In [58]:
gv = 'CONTROL'

In [59]:
plot_earn(dt1, gv)

Loading BokehJS ...

Region

* 0	U.S. Service Schools
* 1	New England (CT, ME, MA, NH, RI, VT)
* 2	Mid East (DE, DC, MD, NJ, NY, PA)
* 3	Great Lakes (IL, IN, MI, OH, WI)
* 4	Plains (IA, KS, MN, MO, NE, ND, SD)
* 5	Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV)
* 6	Southwest (AZ, NM, OK, TX)
* 7	Rocky Mountains (CO, ID, MT, UT, WY)
* 8	Far West (AK, CA, HI, NV, OR, WA)
* 9	Outlying Areas (AS, FM, GU, MH, MP, PR, PW, VI)

Control
* 1	Public
* 2	Private nonprofit
* 3	Private for-profit


CIPDESC - major field

CREDDESC - Text description of the level of credential

CREDLEV
* Undergraduate Certificate or Diploma
* Associate's Degree
* Bachelor’s Degree
* Post-baccalaureate Certificate
* Master's Degree
* Doctoral Degree
* First Professional Degree
* Graduate/Professional Certificate



HIGHDEG

* 0	Non-degree-granting
* 1	Certificate degree
* 2	Associate degree
* 3	Bachelor's degree
* 4	Graduate degree

LOCALE

* 11	City: Large (population of 250,000 or more)
* 12	City: Midsize (population of at least 100,000 but less than 250,000)
* 13	City: Small (population less than 100,000)
* 21	Suburb: Large (outside principal city, in urbanized area with population of 250,000 or more)
* 22	Suburb: Midsize (outside principal city, in urbanized area with population of at least 100,000 but less than 250,000)
* 23	Suburb: Small (outside principal city, in urbanized area with population less than 100,000)
* 31	Town: Fringe (in urban cluster up to 10 miles from an urbanized area)
* 32	Town: Distant (in urban cluster more than 10 miles and up to 35 miles from an urbanized area)
* 33	Town: Remote (in urban cluster more than 35 miles from an urbanized area)
* 41	Rural: Fringe (rural territory up to 5 miles from an urbanized area or up to 2.5 miles from an urban cluster)
* 42	Rural: Distant (rural territory more than 5 miles but up to 25 miles from an urbanized area or more than 2.5 and up to 10 miles from an urban cluster)
* 43	Rural: Remote (rural territory more than 25 miles from an urbanized area and more than 10 miles from an urban cluster)


CCSIZSET
* -2	Not applicable
* 0	(Not classified)
* 1	Two-year, very small
* 2	Two-year, small
* 3	Two-year, medium
* 4	Two-year, large
* 5	Two-year, very large
* 6	Four-year, very small, primarily nonresidential
* 7	Four-year, very small, primarily residential
* 8	Four-year, very small, highly residential
* 9	Four-year, small, primarily nonresidential
* 10	Four-year, small, primarily residential
* 11	Four-year, small, highly residential
* 12	Four-year, medium, primarily nonresidential
* 13	Four-year, medium, primarily residential
* 14	Four-year, medium, highly residential
* 15	Four-year, large, primarily nonresidential
* 16	Four-year, large, primarily residential
* 17	Four-year, large, highly residential
* 18	Exclusively graduate/professional

ADM_RATE_ALL: Admission rate for all campuses rolled up to the 6-digit OPE ID  
SAT_AVG_ALL:Average SAT equivalent score of students admitted for all campuses rolled up to the 6-digit OPE ID

UGDS: Enrollment of undergraduate certificate/degree-seeking students

PFTFAC: Proportion of faculty that is full-time

MEDIAN_HH_INC:
Median household income of students at institution
Data element describes the earnings cohort (without exclusions for military and in-school deferments in the measurement year)


Average family income in real 2015 dollars	student	demographics.avg_family_income	integer	FAMINC

Median family income in real 2015 dollars	student	demographics.median_family_income	integer	MD_FAMINC
